# **Generiranje teksta pomoću mreže s povratnom vezom**

##Učitavanje početnog teksta

Random služi za nasumično odabiranje broja, numpy za matematičke funkcije, a tensorflow za treniranje modela.

In [1]:
import random
import numpy as np
import tensorflow as tf

Učitavamo početni file. Za ovaj projekt sam se odlučio za michael jacksona, američkog pop pjevača. Dokument sadrži tekstove njegovih pijesama u .txt formatu.

In [3]:
filepath = "/content/michael-jackson.txt"
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

##Obrada teksta

Iz teksta uzimamo samo jednom svaki mogući znak i sortiramo ih uzlazno. Nakon toga radimo dva dictionarya, jedan za pretvaranje iz znaka u numeričku vrijednost, a drugi za pretvaranje iz numeričke vrijednosti u znak.

In [4]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

Postavljamo SEQ_LENGTH na 40 što će biti duljina skupa kojem predviđamo koji će biti njegov sljedeći znak, a STEP_SIZE koliko mjesta ćemo se svaki put pomaknuti desno nakon analize pojedinačnog skupa.

In [5]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

For petlja koja pravi rečenice i njihove prave znakove koji dolaze iza njih koje ćemo koristiti za treniranje modela.

In [6]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

Imamo liste i trebmo ih pretvoriti u vektore kako bi ih mogli koristiti za treniranje. 

U ovom slučaju x (ulaz) ima oblik (85322, 40, 66) što znači da imamo 85322 rečenice, svaka duljine 40 i u njoj mogu biti neki od 66 mogućih znakova. Također za y (izlaz) 85322 znakova koji može biti jedan on mogućih 66.

Dalje se nalaze dvije for petlje. Ide se po svakoj rečenici i za svaki znak u rečenici postavi se 1 ili True za taj odgovarajući znak.

In [7]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

print(x.shape)
print(y.shape)

(85322, 40, 66)
(85322, 66)


##Treniranje modela

Koristimo Sequential za naš model, Activation, Dense i LSTM za slojeve, a RMSprop za optimizaciju tijekom kompajliranja.

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

Prvo pozivamo sekvencijalnu metodu te dodajemo LSTM sloj sa 128 neurona ulaznog oblika duljine rečenica puta broj mogućih znakova. Zatim odmah bez skrivenih slojeva dodajemo izlazni sloj Dense sa brojem izlaza koliko ima mogućih znakova. Aktivacijska funkcija "softmax" nam daje vjerovatnosti za svaki izlaz (koliko koji znak je vjerovatan da bude sljedeći). 

In [9]:
model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters), activation="softmax"))

Kompajliramo model pomoću kategoralne unakrsne entropije i fitamo po našim podatcima koje smo prethodno obradili.

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01))

model.fit(x, y, batch_size=128, epochs=40)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/40
667/667 [==============================] - 12s 15ms/step - loss: 0.8544
Epoch 2/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8362
Epoch 3/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8309
Epoch 4/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8236
Epoch 5/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8209
Epoch 6/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8151
Epoch 7/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8144
Epoch 8/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8085
Epoch 9/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8032
Epoch 10/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8086
Epoch 11/40
667/667 [==============================] - 9s 14ms/step - loss: 0.8083
Epoch 12/40
667/667 [==============================] - 9s 14ms/step - loss: 0.7980
Epoch 13/40


##Funkcije

Model nam daje vjerojatnost koji znak bi mogao doći tako da trebamo uzeti vjerojatnost i pretvoriti je u znak što nam pomaže funkcija **sample**. Odabiremo znak sa najvećom vjerovatnosti koju je model predvidio

In [35]:
def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Funkcija prima duljinu koliko želimo da generirani tekst ima znakova. Uzimamo nasumično jedan dio iz cijelog teksta duljine 40 znakova koji će služiti kao početak te na njega dodavati predviđene znakove.

For petlja služi za dodavanje predviđenog znaka.Svaki skup pretvaramo u array kao što smo radili prije treniranja kako bi mogli dobiti rezultat iz modela. 

Pozova se funkcija sample koja vraća znak koji ima najveću vjerovatnost biti sljedeći te ga dodajemo na naš tekst nakon čega uklonimo prvi znak iz skupa koji smo koristili za predviđanje i dodamo novi na kraj te na tom novom skupu predviđamo izlaz.

In [19]:
def generate_text(length):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

##Ispis

**Generiranje teksta**

Pozovemo funkciju generate_text i ispišemo tekst. Dobili smo tekst koji se može čitati, ali nema nekog značenja, prvih 40 znakova je uzeto iz počenog teksta, a ostatak je generiran. Iako rečenice nemaju nekog smisla možemo razaznati većinu riječi što znači da je model dobro istreniran.

In [39]:
print(generate_text(600))

ind a way 
to erase the past 
baby don't make me
baby i need, know you're crys
and i says in the smiliralacing
it's
too bad to re mexlel go

they'd really always usterstay
get your heart
i love that i wanna do
what at's hearched in verion
the girl wall out of make mtl on baby

[chorus:]
oh for me baby, bling
all alone alone, for you
so tears to let it beliby

it's gonna truct be feet)
we can't take you the far,ever, i can't cry
you'll driplow on susting
coming to go
live me all agappher
it's
i make me
i'm bad withoused lows
you got a such you 
year,
you're taken i knees
what arres ary farestion
about about her ence 
